In [1]:
import lkml as looker
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml
import git

from lookml_base_dict import d2

def get_all_values(nested_dictionary):
    
    for key,value in nested_dictionary.items():

        explore = {

            "explore": key,

            "view_name": key
                
            }
            
        
        yield(looker.dump(explore))

        for key,value in value.items():
                    

            join = {

                "joins": [
                    {
                    "sql_on": key[1]+"."+key[2]+" = "+ key[1]+"."+key[0],
                    "name": key[1],
                    "view_name": key
                    }

                ]
            }

            yield(looker.dump(join))


nested_dictionary = d2

get_all_values(nested_dictionary)

def get_git_root(path):

        git_repo = git.Repo(path, search_parent_directories=True)
        git_root = git_repo.git.rev_parse("--show-toplevel")
        return (git_root)

git_def_path = get_git_root(os.getcwd())


def explore_output():
    
    git_path = git_def_path

    rel_path = "base"

    path = os.path.join(git_path, rel_path)

    if not os.path.exists(path):
        os.makedirs(path)
        
    filename = '_explore.layer.lkml'

    with open(os.path.join(path, filename), 'w') as file:

        with redirect_stdout(file):

                for value in get_all_values(nested_dictionary):

                    print(value)

explore_output()
    

In [2]:
join

NameError: name 'join' is not defined

In [3]:
                for value in get_all_values(nested_dictionary):

                    print(value)

explore: wh_website_users_dim
view_name: wh_website_users_dim
join: wh_website_event_pages_fct {
  sql_on: wh_website_event_pages_fct.user_blended_fk = wh_website_event_pages_fct.user_blended_pk ;;
  view_name: [user_blended_pk, wh_website_event_pages_fct, user_blended_fk]
}
join: wh_website_event_tracks_fct {
  sql_on: wh_website_event_tracks_fct.user_blended_fk = wh_website_event_tracks_fct.user_blended_pk ;;
  view_name: [user_blended_pk, wh_website_event_tracks_fct, user_blended_fk]
}


In [4]:
d2

{'wh_website_users_dim': {('user_blended_pk',
   'wh_website_event_pages_fct',
   'user_blended_fk'): {'pk_table_name_value': 'wh_website_users_dim'},
  ('user_blended_pk',
   'wh_website_event_tracks_fct',
   'user_blended_fk'): {'pk_table_name_value': 'wh_website_users_dim'}}}

In [5]:
import lkml as looker
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml

import warehouse_target
import config

import git

pd.options.mode.chained_assignment = None

credentials = config.service_account

warehouse_name = config.warehouse_name
lookml_project = config.project_name

sql = warehouse_target.warehouse_schema
explore_sql = warehouse_target.lookml_explore_schema

if warehouse_name == 'big_query':

    # Run a Standard SQL query with the project set explicitly
    project_id = lookml_project
    df = pandas.read_gbq(sql, dialect='standard', project_id=lookml_project, credentials=credentials)

    df['description'] = df['description'].fillna('not available')

    df1 = df[['table_name','column_name','data_type','description']]

    df1['data_type'] = df1['data_type'].str.replace('TIMESTAMP','timestamp')
    df1['data_type'] = df1['data_type'].str.replace('DATE','date')
    df1['data_type'] = df1['data_type'].str.replace('INT64','number')
    df1['data_type'] = df1['data_type'].str.replace('FLOAT64','number')
    df1['data_type'] = df1['data_type'].str.replace('NUMERIC','number')
    df1['data_type'] = df1['data_type'].str.replace('STRING','string')
    df1['data_type'] = df1['data_type'].str.replace('BOOL','yesno')

    df2 = df1

    explore_df = pandas.read_gbq(explore_sql, dialect='standard', project_id=lookml_project, credentials=credentials)
   

elif warehouse_name == 'snowflake': 

    url = URL(

        account = config.snowflake_account,
        user =  config.snowflake_user,
        schema =  config.snowflake_schema,
        database =  config.snowflake_database,
        password =  config.snowflake_password,
        warehouse= config.snowflake_warehouse,
        role =  config.snowflake_role

    )

    engine = create_engine(url)

    connection = engine.connect()

    query = '''
    select * from snowflake_sample_data.information_schema.columns;
    '''

    df = pd.read_sql(query, connection)
    
    df['description'] = df['comment'].fillna('not available')
    
    df1 = df.groupby(['table_name', 'column_name','data_type','description']).size().reset_index().rename(columns={0:'count'})

    df2 = df1[['table_name','column_name','data_type','description']]

    connection.close()
    engine.dispose()

    
df3 = {n: grp.loc[n].to_dict('index')
    
for n, grp in df2.set_index(['table_name', 'column_name','data_type','description']).groupby(level='table_name')}

d1 = df3

##

df4 = {n: grp.loc[n].to_dict('index')
    
for n, grp in explore_df.set_index(['pk_table_name', 'pk_column_name','fk_table_name','fk_column_name']).groupby(level='pk_table_name')}

d2 = df4

##

def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.iloc[:,1:]) for k,g in grouped}
    return d

d5 = (recur_dictify(df1))

				

In [6]:
d5


{'wh_website_event_pages_fct': {'device': {'string': 'Derived device description'},
  'device_category': {'string': 'Device category derived from device'},
  'event_pages_pk': {'string': 'A unique identifier for each page view event'},
  'event_pages_ts': {'timestamp': 'Timestamp when the datapoint occured, corrected for client-device clock skew.'},
  'google_click_natural_key': {'string': 'Google Ads click id (gclid)'},
  'page_title': {'string': 'Webpage title'},
  'page_url_host': {'string': 'Webpage URL host address'},
  'page_url_path': {'string': 'Webpage URL file path'},
  'page_url_query': {'string': 'Webpage URL query'},
  'received_at_ts': {'timestamp': 'Time on Segment server clock when call was received'},
  'referrer': {'string': 'Information about how the user was referred to the website'},
  'referrer_host': {'string': 'Referrers host address'},
  'search_params': {'string': 'Search parameters'},
  'segment_identify_natural_key': {'string': 'Segment Identify Call ID'},
 